# File to analyze graphics of default simulation
------------------------------------------------

## Imports to collect data and plot graphics

In [ ]:
from components.graphic_generator import Graphic_Gen
from components.data_manager import Data_Manager
from components.utils.enums import Topologies
import components.network_data as nwd
from components.utils.enums import Colors
import numpy as np
import pandas as pd

## Constants

In [ ]:
PATH_TO_CSV: str = "data/topology_simulation/topology_simulation.csv"
NUMBER_OF_NODES: tuple = tuple([i for i in range(12, 108, 12)])
PARAMS_TUPLE_ERDOS_RENYI: tuple[float, float, float] = (0.1, 0.3, 0.5)
PARAMS_TUPLE_BARABASI_ALBERT: tuple[int, int, int] = (1, 3, 5)
STANDARD_DEVIATION: bool = False
GRID: bool = True
COLORS_TUPLE: tuple[Colors, Colors, Colors] = (Colors.RED, Colors.ORANGE, Colors.YELLOW)

## Load csv

In [ ]:
dm: Data_Manager = Data_Manager()
loaded: bool = dm.load_csv(PATH_TO_CSV)
if not loaded:
    print("File don't loaded")

## Convert csv to pandas DataFrame

In [ ]:
dframe: pd.DataFrame = dm.csv_to_dataframe()

## Create a function to collect default networks simulations

In [ ]:
def collect_default_grid_arrays(
    df: pd.DataFrame,
    desired_column: str,
    multiply_factor: int = 1,
    divisor_column: str | None = None
) -> np.ndarray:
    """
    Create 2D arrays with data of default grid networks.

    Args:
        df (DataFrame): DataFrame with all data storage
        desired_column (str): Name of column desired
        multiply_factor (int): Number to multiply all lines
        divisor_column (str | None): Name of column to divide the arrays. If is None don't division occurs

    Returns:
        ndarray[ndarray]: Line of array is a array to calculate the point of graphic
    """

    list_of_arrays: list[np.ndarray] = []

    for number_of_nodes in NUMBER_OF_NODES:
        mask: pd.Series = (
            (df[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
            (df[nwd.TOPOLOGY] == Topologies.GRID.value) & 
            (df[nwd.NUMBER_OF_NODES] == number_of_nodes)
        )

        if divisor_column is None:
            data: np.ndarray = (df.loc[mask, desired_column] * multiply_factor).to_numpy()
        else:
            data: np.ndarray = ((df.loc[mask, desired_column] / df.loc[mask, divisor_column]) * multiply_factor).to_numpy()
        
        list_of_arrays.append(data)

    return np.array(list_of_arrays)


def collect_default_arrays_dict(
    df: pd.DataFrame,
    topology: Topologies,
    parameters_columns: list[int | float],
    desired_column: str,
    multiply_factor: int = 1,
    divisor_column: str | None = None
) -> dict[int | float, np.ndarray]:
    """
    Create 2D arrays with data of default networks to use in Graphic Generator.

    Args:
        df (DataFrame): DataFrame with all data storage
        topology (str): Name of the desired topology
        parameters_columns (list[str]): Ordened list with all parameters
        desired_column (str): Name of column desired
        multiply_factor (int): Number to multiply all lines
        divisor_column (str | None): Name of column to divide the arrays. If is None don't division occurs

    Returns:
        dict[str, ndarray[ndarray]]: Dict with key is the parameter and line of array is a array to calculate the point of graphic
    """

    dict_of_arrays: dict[int | float, np.ndarray] = dict()
    
    for parameter in parameters_columns:
        
        list_of_arrays: list[np.ndarray] = []

        for number_of_nodes in NUMBER_OF_NODES:
            mask: pd.Series = (
                (df[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
                (df[nwd.TOPOLOGY] == topology.value) & 
                (df[nwd.NUMBER_OF_NODES] == number_of_nodes) &
                (df[nwd.PARAMETER] == parameter)
            )

            if divisor_column is None:
                data: np.ndarray = (df.loc[mask, desired_column] * multiply_factor).to_numpy()
            else:
                data: np.ndarray = ((df.loc[mask, desired_column] / df.loc[mask, divisor_column]) * multiply_factor).to_numpy()
            
            list_of_arrays.append(data)
            
        dict_of_arrays[parameter] = np.array(list_of_arrays)

    return dict_of_arrays

## Manipulate data to collect columns from default networks (without black holes)

In [ ]:
# ===============
#      Grid
# ===============

# 2D arrays
# line: array[i] = array to converte a point of graphic

default_grid_consummed_eprs_array: np.ndarray = collect_default_grid_arrays(
    df=dframe,
    desired_column=nwd.CONSUMED_EPRS
)

# (total success / total requests) * 100
default_grid_success_tax_array: np.ndarray = collect_default_grid_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    divisor_column=nwd.REQUESTS,
    multiply_factor=100
)

# total fidelity / success requests
default_grid_avg_fidelity_array: np.ndarray = collect_default_grid_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)


# ===============
# Barabási-Albert
# ===============

default_barabasi_albert_consummed_eprs_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.CONSUMED_EPRS
)

# (total success / total requests) * 100
default_barabasi_albert_success_tax_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    divisor_column=nwd.REQUESTS,
    multiply_factor=100
)

# total fidelity / success requests
default_barabasi_albert_avg_fidelity_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)


# ===============
#   Erdős-Rényi
# ===============

default_erdos_renyi_consummed_eprs_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.CONSUMED_EPRS
)

# (total success / total requests) * 100
default_erdos_renyi_success_tax_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    divisor_column=nwd.REQUESTS,
    multiply_factor=100
)

# total fidelity / success requests
default_erdos_renyi_avg_fidelity_dict: dict[int | float, np.ndarray] = collect_default_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

## Functions to collect attacked networks

In [ ]:
def collect_attacked_grid_networks_arrays(
    df: pd.DataFrame, 
    desired_column: str,
    targets: int,
    multiply_factor: int = 1,
    divisor_column: str | None = None
) -> np.ndarray:
    """
    Create 2D arrays with data of attacked grid networks to plot graphics in Graphic Generator.

    Args:
        df (DataFrame): DataFrame with all data storage
        desired_column (str): Name of column desired
        targets (int): Number of targets per black hole
        multiply_factor (int): Number to multiply all lines
        divisor_column (str | None): Name of column to divide the arrays. If is None don't division occurs

    Returns:
        ndarray[ndarray]: Line of array is a array to calculate the point of graphic
    """
    
    list_of_arrays: list[np.ndarray] = list()

    for number_of_nodes in NUMBER_OF_NODES:

        mask: pd.Series = (
            (df[nwd.TOPOLOGY] == Topologies.GRID.value) &
            (df[nwd.NUMBER_OF_BLACK_HOLES] != 0) &
            (df[nwd.NUMBER_OF_NODES] == number_of_nodes) &
            (df[nwd.TARGETS_PER_BLACK_HOLE] == targets)
        )

        if divisor_column is None:
            data: np.ndarray = (df.loc[mask, desired_column] * multiply_factor).to_numpy()
        else:
            data: np.ndarray = ((df.loc[mask, desired_column] / df.loc[mask, divisor_column]) * multiply_factor).to_numpy()
        
        list_of_arrays.append(data)

    return np.array(list_of_arrays)


def collect_attacked_networks_arrays_dict(
    df: pd.DataFrame,
    topology: Topologies, 
    parameters_columns: list[int | float],
    desired_column: str,
    targets: int,
    multiply_factor: int = 1,
    divisor_column: str | None = None  
) -> dict[int | float, np.ndarray]:
    """
    Create 2D arrays with data of attacked networks to use in Graphic Generator.

    Args:
        df (DataFrame): DataFrame with all data storage
        topology (str): Name of the desired topology
        parameters_columns (list[str]): Ordened list with all parameters
        desired_column (str): Name of column desired
        targets (int): Number of targets per black hole
        multiply_factor (int): Number to multiply all lines
        divisor_column (str | None): Name of column to divide the arrays. If is None don't division occurs

    Returns:
        dict[str, ndarray[ndarray]]: Dict with key is the parameter and line of array is a array to calculate the point of graphic
    """
    
    dict_of_arrays: dict[int | float, np.ndarray] = dict()

    for parameter in parameters_columns:
        list_of_points: list[np.ndarray] = list()
        for number_of_nodes in NUMBER_OF_NODES:
            mask: pd.Series = (
                (df[nwd.TOPOLOGY] == topology.value) &
                (df[nwd.NUMBER_OF_BLACK_HOLES] != 0) &
                (df[nwd.TARGETS_PER_BLACK_HOLE] == targets) &
                (df[nwd.PARAMETER] == parameter) &
                (df[nwd.NUMBER_OF_NODES] == number_of_nodes)
            )

            if divisor_column is None:
                data: np.ndarray = (df.loc[mask, desired_column] * multiply_factor).to_numpy()
            else:
                data: np.ndarray = ((df.loc[mask, desired_column] / df.loc[mask, divisor_column]) * multiply_factor).to_numpy()
            
            list_of_points.append(data)
        dict_of_arrays[parameter] = np.array(list_of_points)

    return dict_of_arrays

## Collect all data from Grid attacked networks

In [ ]:
# ===============
#      GRID
# ===============

attacked_grid_consummed_eprs_no_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.CONSUMED_EPRS,
    targets=0,
)

attacked_grid_consummed_eprs_with_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.CONSUMED_EPRS,
    targets=1,
)

attacked_grid_success_tax_no_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=0,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS,
)

attacked_grid_success_tax_with_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=1,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS
)

attacked_grid_avg_fidelity_no_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    targets=0,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

attacked_grid_avg_fidelity_with_targets_axis: np.ndarray = collect_attacked_grid_networks_arrays(
    df=dframe,
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=1,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

## Collect all data from Barabási-Albert attacked networks

In [ ]:
# ===============
# Barabási-Albert
# ===============

attacked_barabasi_albert_consummed_eprs_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.CONSUMED_EPRS,
    targets=0,
)

attacked_barabasi_albert_consummed_eprs_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.CONSUMED_EPRS,
    targets=1,
)


attacked_barabasi_albert_success_tax_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=0,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS
)

attacked_barabasi_albert_success_tax_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=1,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS
)

attacked_barabasi_albert_avg_fidelity_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    targets=0,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

attacked_barabasi_albert_avg_fidelity_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.BARABASI_ALBERT,
    parameters_columns=[i for i in PARAMS_TUPLE_BARABASI_ALBERT],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    targets=1,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

## Collect all data from Erdős-Rényi attacked networks

In [ ]:
# ===============
#   Erdős-Rényi
# ===============

attacked_erdos_renyi_consummed_eprs_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.CONSUMED_EPRS,
    targets=0,
)

attacked_erdos_renyi_consummed_eprs_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.CONSUMED_EPRS,
    targets=1,
)


attacked_erdos_renyi_success_tax_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=0,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS
)

attacked_erdos_renyi_success_tax_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    targets=1,
    multiply_factor=100,
    divisor_column=nwd.REQUESTS
)

attacked_erdos_renyi_avg_fidelity_no_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    targets=0,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

attacked_erdos_renyi_avg_fidelity_with_targets_dict: dict[int | float, np.ndarray] = collect_attacked_networks_arrays_dict(
    df=dframe,
    topology=Topologies.ERDOS_RENYI,
    parameters_columns=[i for i in PARAMS_TUPLE_ERDOS_RENYI],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    targets=1,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)

------------------------
# Grid topology graphics
------------------------

## Success Tax

### Grid without targets Success Tax

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_success_tax_no_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_success_tax_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Success Tax (target off)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)

### Grid with targets Success Tax

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_success_tax_with_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_success_tax_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Success Tax (target off)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)

## Consummed EPRs

### Grid without targets consummed EPRs

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_consummed_eprs_no_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_consummed_eprs_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Consummed EPRs (target off)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)

### Grid with targets consummed EPRs

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_consummed_eprs_with_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_consummed_eprs_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Consummed EPRs (target on)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)

## Average Route Fidelity

### Grid without targets Average Route Fidelity

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_avg_fidelity_no_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_avg_fidelity_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Route Fidelity (target off)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)

### Grid with targets Average Route Fidelity

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))
graphic_generator.append_y_axle(
    axle=attacked_grid_avg_fidelity_with_targets_axis,
    label='Grid',
    standard_deviation=STANDARD_DEVIATION,
    default_axle=default_grid_avg_fidelity_array,
    color=Colors.RED)

graphic_generator.plot(tittle="Grid Route Fidelity (target on)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)

-------------------------------
# Erdős-Rényi topology graphics
-------------------------------

## Success Tax

### Erdős-Rényi without targets Success Tax

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_success_tax_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_success_tax_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Success Tax (target off)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)


### Erdős-Rényi with targets Success Tax

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_success_tax_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_success_tax_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Success Tax (target on)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)


## Consummed EPRs

### Erdős-Rényi without targets Consummed EPRs

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_consummed_eprs_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_consummed_eprs_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Consummed EPRs (target off)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)


### Erdős-Rényi with targets Consummed EPRs

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_consummed_eprs_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_consummed_eprs_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Consummed EPRs (target on)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)


## Average Route Fidelity

### Erdős-Rényi without targets Average Route Fidelity

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_avg_fidelity_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_avg_fidelity_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Route Fidelity (target off)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)


### Erdős-Rényi with targets Average Route Fidelity

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_ERDOS_RENYI, COLORS_TUPLE):
    param_key: str = f"p={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_erdos_renyi_avg_fidelity_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_erdos_renyi_avg_fidelity_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Erdős-Rényi Route Fidelity (target on)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)


----------------------------------
# Barbási-Albert topology graphics
----------------------------------

## Success Tax

### Barbási-Albert without targets Success Tax

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_success_tax_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_success_tax_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Success Tax (target off)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)


### Barbási-Albert with targets Success Tax

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_success_tax_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_success_tax_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Success Tax (target on)", x_label="Number of nodes", y_label="Diff Success Tax", grid=GRID)


## Consummed EPRs

### Barbási-Albert without targets Consummed EPRs

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_consummed_eprs_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_consummed_eprs_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Consummed EPRs (target off)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)


### Barabási-Albert with targets Consummed EPRs

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_consummed_eprs_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_consummed_eprs_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Consummed EPRs (target on)", x_label="Number of nodes", y_label="Diff Consummed EPRs", grid=GRID)


## Average Route Fidelity

### Barabási-Albert without targets Average Route Fidelity

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_avg_fidelity_no_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_avg_fidelity_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Route Fidelity (target off)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)


### Erdős-Rényi with targets Average Route Fidelity

In [ ]:
graphic_generator = Graphic_Gen()
graphic_generator.define_x_axle(np.array(NUMBER_OF_NODES))

for parameter, color in zip(PARAMS_TUPLE_BARABASI_ALBERT, COLORS_TUPLE):
    param_key: str = f"m={parameter}"
    graphic_generator.append_y_axle(
        axle=attacked_barabasi_albert_avg_fidelity_with_targets_dict[parameter],
        label=param_key,
        standard_deviation=STANDARD_DEVIATION,
        default_axle=default_barabasi_albert_avg_fidelity_dict[parameter],
        color=color
    )

graphic_generator.plot(tittle="Barabási-Albert Route Fidelity (target on)", x_label="Number of nodes", y_label="Diff Route Fidelity", grid=GRID)


# Correlation Graphic

In [ ]:
df: pd.DataFrame = dframe.drop(
    columns=[
        nwd.INTENSITY,
        nwd.NORMAL_NODE_SWAP_PROB,
        nwd.REQUESTS
    ]
) # remove constants columns and string columns

Graphic_Gen.plot_heatmap(df.corr(), title="Heatmap of Correlation in Default Simulations")